In [460]:
import pandas as pd
import numpy as np
import itertools as it
import networkx as nx
# Documentacion de la libreria: http://networkx.readthedocs.io/en/networkx-1.11/

In [461]:
def filterData(df, isTraining, date):
    """
        Funcion que devuelve el conjunto de problemas que tienen status AC o PE
        Si isTraining es true, entonces la funcion sacara el training_set, si no, sacara el evaluation_set
        date es la fecha de particion
    """
    
    if isTraining:
        df = df[df['submissionDate'] < date]
        df = df.loc[df['status'].isin(['AC', 'PE'])]
    else:
        df = df[df['submissionDate'] >= date]
    
    

    return df

In [462]:
# MAIN
# ---------

# se guarda en la variable df (DataFrame) toda la base de datos
df = pd.read_csv('bbdd_orderbydate.csv')

# aqui quito los problemas que no existian despues de la fecha umbral
df = df[df['problem_id'] <= 511] 

# construyo el conjunto de entrenamiento
training_set = filterData(df, True, "2016-10-21 00:00:00")

print(training_set)

# obtengo los nodos del grafo, esta vez los nodos son los usuarios y no los problemas:
nodes = training_set.user_id.unique()

# creo un diccionario que va a tener a los usuarios como keys y los valores seran los
# problemas que ha hecho ese usuario
grouped = training_set.groupby('user_id')['problem_id'].apply(list)

# muestra el numero de usuarios
print(len(nodes))

# muestra la lista de problemas que ha hecho cada usuario
print(grouped)

       problem_id  user_id status       submissionDate
0              10        5     AC  2014-02-17 15:27:07
1               2        6     AC  2014-02-17 15:39:17
2               2        9     AC  2014-02-18 00:30:14
3              10        9     AC  2014-02-18 00:34:46
4               4        9     AC  2014-02-18 00:50:28
5               6        9     AC  2014-02-18 00:52:11
6              13        9     AC  2014-02-18 00:53:40
9              15        8     AC  2014-02-19 19:58:03
10              4        8     AC  2014-02-20 14:23:30
11             39       16     AC  2014-02-20 15:44:33
12             39       17     AC  2014-02-20 16:53:34
13             13       12     AC  2014-02-21 11:08:38
15             33       12     AC  2014-02-21 11:58:32
16             39       12     AC  2014-02-21 12:05:49
17             44       12     AC  2014-02-21 12:52:44
18             44       15     AC  2014-02-21 14:51:52
19             44       22     AC  2014-02-21 14:58:55
21        

In [463]:
print(training_set)

       problem_id  user_id status       submissionDate
0              10        5     AC  2014-02-17 15:27:07
1               2        6     AC  2014-02-17 15:39:17
2               2        9     AC  2014-02-18 00:30:14
3              10        9     AC  2014-02-18 00:34:46
4               4        9     AC  2014-02-18 00:50:28
5               6        9     AC  2014-02-18 00:52:11
6              13        9     AC  2014-02-18 00:53:40
9              15        8     AC  2014-02-19 19:58:03
10              4        8     AC  2014-02-20 14:23:30
11             39       16     AC  2014-02-20 15:44:33
12             39       17     AC  2014-02-20 16:53:34
13             13       12     AC  2014-02-21 11:08:38
15             33       12     AC  2014-02-21 11:58:32
16             39       12     AC  2014-02-21 12:05:49
17             44       12     AC  2014-02-21 12:52:44
18             44       15     AC  2014-02-21 14:51:52
19             44       22     AC  2014-02-21 14:58:55
21        

In [464]:
# OBTENCION DEL EVALUATION_SET
# -------

# ahota saco el evaluation_set
evaluation_set = filterData(df, False, "2016-10-21 00:00:00")

print(evaluation_set)

# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user_eval = evaluation_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users_eval = pd.DataFrame({'user_id':grouped_user_eval.index, 'list_problem_id':grouped_user_eval.values})

print(df_users_eval)

       problem_id  user_id status       submissionDate
16939         469      799     RF  2016-10-21 00:08:23
16940         325     3832     AC  2016-10-21 00:08:35
16942         469     3757     AC  2016-10-21 00:41:36
16943         469     3792    RTE  2016-10-21 01:04:08
16944         469      810     AC  2016-10-21 01:48:35
16945         438     3943     AC  2016-10-21 02:23:56
16946         469     3773     AC  2016-10-21 05:00:58
16947         346     3728     AC  2016-10-21 09:06:38
16948         469     3736     CE  2016-10-21 09:38:14
16949         438     3785     TL  2016-10-21 09:43:52
16950         469     3704     AC  2016-10-21 10:04:03
16951         469     2912     AC  2016-10-21 10:20:36
16952         213     3136     AC  2016-10-21 10:28:23
16953         213     3088     AC  2016-10-21 10:29:26
16954         469     3708    RTE  2016-10-21 10:35:15
16956         235     3136    RTE  2016-10-21 10:37:58
16957         183     3880     AC  2016-10-21 11:38:11
16958     

In [465]:
# In[3]:

def compareNodes(f_list, s_list):
    """
        Funcion que devuelve el numero de usuarios que han hecho ambos problemas
    """
    peso = len(np.intersect1d(f_list, s_list))
    
    return peso
    
def createLinks(prob_us_set, nodos):
    """
        Funcion que crea los enlaces del grafo a partir de la informacion contenida en el conjunto que se le
        pasa a la funcion
    """
    resultado = list() 
    
    # hago todas las posibles combinaciones de usuarios
    for fst, snd in it.combinations(nodos, 2):
        # obtengo el peso pasando la lista de problemas que ha hecho cada usuario
        peso = compareNodes(prob_us_set[fst], prob_us_set[snd])
        if peso >= 1:
            resultado.append((fst, snd, peso))
            
            
            
    return resultado


# In[4]:

def filterWeight(weightUmbral, linksToFilter):
    """
        Funcion que filtra los enlaces de un grafo, para que el peso sea mayor o igual al dado
    """
    
    result = [(x, y, z) for (x, y, z) in linksToFilter if z >= weightUmbral]
    
    return result
    


# In[5]:

def create_graph_nx(list_nodes, list_links):
    """
        Funcion que crea un grafo de tipo Graph de la libreria NetworkX
        Construccion del grafo: http://networkx.readthedocs.io/en/networkx-1.11/tutorial/tutorial.html#what-to-use-as-nodes-and-edges
    """
    grafo = nx.Graph() # creo la variable grafo

    # incluyo los nodos del grafo 
    grafo.add_nodes_from(list_nodes)

    # se incluyen las tuplas de enlaces con el peso del enlace
    # es una lista de la forma [(Nodo1, Nodo2, peso), ......]
    grafo.add_weighted_edges_from(list_links)

    return grafo


In [466]:
# creo los enlaces a partir de la informacion de los nodos
links = createLinks(grouped, nodes)
# ahora filtro el grafo para que los enlaces solo tengan el peso que quiero
linksFiltered = filterWeight(5, links)

print(len(linksFiltered))

# aqui creo el grafo 
graph = create_graph_nx(nodes, linksFiltered)

16805


In [467]:
def lenProblemsDone(row, set_filter):
    """
        Funcion auxiliar que calcula cuanto problemas ha hecho cada usuario en un conjunto: training o evaluation
    """
    # saco el dataframe que contendra solo una fila con la lista de problemas que ha hecho el usuario
    df_filter = set_filter[set_filter['user_id'] == row['user_id']]
    
    if df_filter.empty:
        # si esta vacio, entonces es que el usuario no ha hecho problemas en ese conjunto
        return 0
    else:
        # sino, devuelvo la longitud de la lista de problemas
        return len(df_filter['list_problem_id'].iloc[0]) 
    

In [468]:
# aqui voy a hacer el filtro de usuarios de forma que para hacer las recomendaciones solo tengamos en 
# cuenta aquellos usuarios que han hecho 5 o mas problemas tanto antes de la fecha limite como despues

# convierto la serie en un dataframe
df_users = pd.DataFrame({'user_id':grouped.index, 'list_problem_id':grouped.values})

# primero guardo la lista de usuarios
user_list = df.user_id.unique()

# la meto en un dataframe 
column_user_filter = {'user_id': user_list}
datraframe_user_filter = pd.DataFrame.from_dict(column_user_filter)


# ahora tengo que calcular para cada fila, el numero de problemas que han hecho en el training_set, evaluation_set
datraframe_user_filter['len_training'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users), axis=1)
datraframe_user_filter['len_evaluation'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users_eval), axis=1)
print(datraframe_user_filter)

      user_id  len_training  len_evaluation
0           5             4               1
1           6             2               0
2           9            11               3
3           8            47               0
4          16             1               0
5          17             2               2
6          12            12               0
7          18             5               8
8          15             3               1
9          22             1               0
10          7             7               0
11         27             0               0
12         28             1               0
13         29             1               0
14         32             2               0
15         35            41               1
16         33             9               0
17         39            11               0
18         31             9               0
19         40             1               0
20         44             8               0
21         48             0     

In [469]:
# ahora tengo que hacer el filtro en este dataframe, de forma que solo aparezcan las filas en las que len_training y 
# len_evaluation sea >=5
datraframe_user_filter = datraframe_user_filter[(datraframe_user_filter['len_training'] >= 5) & (datraframe_user_filter['len_evaluation'] >=5)]
print(datraframe_user_filter)

# aqui voy a guardar la lista de usuarios a los que voy a recomendar
user_list_to_recommend = sorted(datraframe_user_filter['user_id'].tolist())
print(user_list_to_recommend)

      user_id  len_training  len_evaluation
7          18             5               8
29         60            18              14
118        25            22               7
261       130            80               6
299       414            49              61
316       443            53               8
317       448            43              21
483       689            33               7
619       912            13              11
633       935           127               9
1196     1711           128              16
1288     1893             7               6
1311     1952            19               9
1339     1955             9               5
1416     2038             5               5
1423     2096            12              13
1446     2051            11               8
1447     2025            12               8
1448     2120             7               8
1451     2041             9               7
1528     2257             6              13
1694     2576             5     

In [470]:
# ahora tengo que filtrar df_users para que solo contenga las filas en las que los usuarios
# pertenecen a la anterior lista

df_users_recommend = df_users[df_users['user_id'].isin(user_list_to_recommend)]
print(df_users_recommend)

                                        list_problem_id  user_id
11                            [119, 114, 316, 507, 506]       18
14    [258, 141, 309, 310, 390, 187, 510, 438, 13, 2...       25
39    [100, 62, 2, 57, 53, 23, 134, 15, 147, 233, 24...       60
90    [309, 150, 183, 237, 191, 187, 39, 209, 70, 15...      130
241   [213, 2, 109, 114, 10, 436, 438, 437, 404, 239...      414
255   [39, 100, 247, 150, 183, 44, 471, 109, 307, 30...      443
257   [255, 257, 314, 311, 315, 310, 282, 243, 275, ...      448
392   [171, 272, 282, 209, 373, 155, 2, 39, 134, 471...      689
512   [241, 269, 2, 340, 65, 379, 316, 275, 309, 259...      912
523   [65, 241, 239, 105, 231, 269, 275, 331, 257, 1...      935
933   [507, 33, 83, 217, 312, 256, 49, 4, 503, 254, ...     1711
997                      [39, 49, 465, 159, 134, 13, 2]     1893
1021  [325, 441, 237, 183, 438, 510, 226, 4, 327, 21...     1952
1024      [325, 327, 441, 468, 471, 503, 226, 109, 446]     1955
1066  [155, 505, 49, 162,

In [471]:

# en df_new tengo los usuarios a los que tengo que hacer recomendaciones

# primero guardo la lista de usuarios
user_list_recomend = df_users_recommend.user_id.unique()

# creo el nuevo dataframe con los resultados 
column_user_recomend = {'user_id': user_list_to_recommend}
dataframe_user_recomend = pd.DataFrame.from_dict(column_user_recomend)

print(dataframe_user_recomend)

    user_id
0        18
1        25
2        60
3       130
4       414
5       443
6       448
7       689
8       912
9       935
10     1711
11     1893
12     1952
13     1955
14     2025
15     2027
16     2038
17     2041
18     2051
19     2096
20     2106
21     2120
22     2257
23     2355
24     2373
25     2576
26     2626
27     2906
28     2966
29     3118
..      ...
35     3708
36     3710
37     3712
38     3714
39     3722
40     3724
41     3727
42     3731
43     3735
44     3749
45     3759
46     3779
47     3784
48     3795
49     3842
50     3846
51     3847
52     3855
53     3859
54     3873
55     3880
56     3890
57     3931
58     3937
59     3938
60     3941
61     3946
62     3961
63     3963
64     3969

[65 rows x 1 columns]


In [472]:
def apply_wcn(row, graph):
    """
        Funcion que devuelve el valor wcn en comun de esos dos nodos
    """
    cn_list = nx.common_neighbors(graph, row['one'], row['two'])
    
    value_wcn = sum([graph[row['one']][x]['weight'] + graph[row['two']][x]['weight'] for x in cn_list])
    
    return value_wcn

def create_wcn_data(graph, nodes):

    # Ahora voy a construir un DataFrame que tenga dos columnas con todas las posibles combinaciones de problemas, y otra 
    # columna con el valor de wcn para ese par de problemas
    fst_column = list()
    snd_column = list()
    for fst, snd in it.combinations(nodes, 2):
        fst_column.append(fst)
        snd_column.append(snd)

    d = {'one' : fst_column,
        'two' : snd_column}
    dataFrame_wcn = pd.DataFrame(d)


    # Aplico la funcion a cada fila
    dataFrame_wcn['wcn'] = dataFrame_wcn.apply (lambda row: apply_wcn(row, graph), axis=1)


    return dataFrame_wcn

In [473]:
wcn_df = create_wcn_data(graph, nodes)
print(wcn_df)

          one   two  wcn
0           5     6    0
1           5     9    0
2           5     8    0
3           5    16    0
4           5    17    0
5           5    12    0
6           5    15    0
7           5    22    0
8           5     7    0
9           5    28    0
10          5    29    0
11          5    32    0
12          5    35    0
13          5    33    0
14          5    31    0
15          5    40    0
16          5    49    0
17          5    42    0
18          5    46    0
19          5    24    0
20          5    53    0
21          5    60    0
22          5    59    0
23          5    66    0
24          5    68    0
25          5    71    0
26          5    69    0
27          5    70    0
28          5    65    0
29          5    73    0
...       ...   ...  ...
2012991  3699  2046    0
2012992  3699  4018    0
2012993  3997  4011    0
2012994  3997  3951    0
2012995  3997  3949    0
2012996  3997  3952    0
2012997  3997  3982    0
2012998  3997  2046    0


In [474]:
def getCommonNeighbors(row, wcn_df):
    """
        Funcion que devuelve la lista de los usuarios de ese usuario que tienen vecinos en comun
    """
    # print(row['user_id'])
    
    # obtengo dos df con los usuarios que tienen usuarios vecinos con el usuario de la fila 
    column_result_one_tmp = wcn_df[wcn_df['one'] == row['user_id']]
    column_result_one = column_result_one_tmp[column_result_one_tmp['wcn'] > 0]
    column_result_two_tmp = wcn_df[wcn_df['two'] == row['user_id']]
    column_result_two = column_result_two_tmp[column_result_two_tmp['wcn'] > 0]
    
    # saco las listas de usuarios con usuarios comunes
    list_one = list(column_result_one['two'])
    list_two = list(column_result_two['one'])
    
    # la concateno sin tener en cuenta repeticiones, porque nunca va a haber
    list_wcn = list_one + list_two
    
    # print(list_wcn)
    
    if list_wcn == []: # sino tiene vecinos en comun, pongo toda la lista de nodos
        list_wcn = graph.nodes()
        list_wcn.remove(row['user_id']) # y elimino el nodo que estoy mirando
    
    # hago el filtro de los k mejores
    return list_wcn

In [475]:
dataframe_user_recomend['neighbors'] = dataframe_user_recomend.apply (lambda row: getCommonNeighbors(row, wcn_df), axis=1)

# aqui tengo la lista de usuarios con sus k usuarios similares
print(dataframe_user_recomend)

    user_id                                          neighbors
0        18  [3373, 3403, 3422, 3427, 3432, 3440, 3452, 342...
1        25  [418, 437, 440, 443, 448, 445, 302, 454, 130, ...
2        60  [67, 64, 84, 89, 91, 54, 95, 82, 112, 120, 124...
3       130  [461, 396, 483, 103, 488, 416, 509, 511, 527, ...
4       414  [176, 25, 418, 437, 440, 443, 448, 445, 302, 4...
5       443  [448, 445, 302, 454, 130, 461, 396, 483, 103, ...
6       448  [445, 302, 454, 130, 461, 396, 483, 103, 488, ...
7       689  [709, 713, 676, 720, 724, 758, 771, 712, 722, ...
8       912  [840, 919, 864, 930, 933, 813, 934, 935, 105, ...
9       935  [105, 946, 871, 896, 975, 967, 979, 982, 983, ...
10     1711  [1433, 1619, 1733, 636, 1736, 1746, 1751, 1594...
11     1893  [1910, 1919, 1574, 1911, 1921, 1922, 1904, 189...
12     1952  [1962, 1931, 619, 1963, 1961, 1969, 1957, 1983...
13     1955  [1975, 1981, 2050, 2029, 2033, 2034, 2035, 204...
14     2025  [2120, 2053, 2041, 2086, 2095, 2168, 2169,

In [476]:
# ahora voy a separar cada user-problema_a_recomendar para hacer la cuenta
# creo un nuevo dataframe que agrupa por el primer problema y tiene su posible recomendacion
df_separation = dataframe_user_recomend.groupby(['user_id']).neighbors.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

df_separation.columns = ['user_id', 'neighbors']

print(df_separation)

       user_id  neighbors
0           18       3373
1           18       3403
2           18       3422
3           18       3427
4           18       3432
5           18       3440
6           18       3452
7           18       3428
8           18       3481
9           18       3485
10          18       3493
11          18       3498
12          18       3496
13          18       3499
14          18       3495
15          18       3501
16          18       3517
17          18       3550
18          18       3497
19          18       2023
20          18       3558
21          18       3583
22          18       3585
23          18       3492
24          18       3582
25          18       3509
26          18       3601
27          18       2888
28          18       3705
29          18       3706
...        ...        ...
37587     3969       3847
37588     3969       3859
37589     3969       3862
37590     3969       2040
37591     3969       3845
37592     3969       3854
37593     39

In [477]:
def getProblemsFromSimilarUSers(row, df_users, df_users_recommend):
    """
        Funcion que va a devolver por cada fila una lista procedente de problemas que han
        hecho los usuarios similares a ese. Además eliminara los problemas que ya haya hecho el usuario
    """
    
    # obtengo la lista de problemas que ha hecho el usuario en cuestion
    list_problems_users = df_users_recommend[df_users_recommend['user_id'] == row['user_id']]
    list_problems_user = list(list_problems_users['list_problem_id'])[0]
    
    # aqui saco la lista de problemas que ha hecho el usuario similar
    list_problems_df = df_users[df_users['user_id'] == row['neighbors']]    
    lista_problemas_comprobar = list(list_problems_df['list_problem_id'])[0]
    
    

    # aqui hago el filtro para que no se incluyan los problemas que ya ha hecho el usuario
    list_problems = [x for x in lista_problemas_comprobar if x not in list_problems_user]
    
    return list_problems

In [478]:
# ahora para cada lista de de usuarios, hacer una lista de los problemas realizados por esos usuarios, 
# que no los haya realizado ya el usuario
df_separation['list_problems'] = df_separation.apply (lambda row: getProblemsFromSimilarUSers(row, df_users, df_users_recommend), axis=1)

print(df_separation)

       user_id  neighbors                                      list_problems
0           18       3373                      [62, 502, 509, 105, 307, 166]
1           18       3403         [39, 2, 6, 8, 10, 13, 33, 139, 35, 44, 49]
2           18       3422                 [39, 150, 183, 327, 191, 325, 390]
3           18       3427       [150, 254, 134, 191, 325, 390, 39, 162, 327]
4           18       3432                      [134, 109, 150, 39, 327, 325]
5           18       3440                           [134, 150, 39, 327, 325]
6           18       3452                              [39, 44, 33, 49, 253]
7           18       3428  [150, 78, 171, 181, 51, 155, 174, 136, 235, 44...
8           18       3481  [39, 33, 109, 13, 162, 134, 159, 251, 27, 29, ...
9           18       3485                    [162, 39, 33, 44, 49, 254, 183]
10          18       3493   [109, 39, 70, 141, 134, 187, 258, 150, 162, 209]
11          18       3498                         [44, 39, 33, 70, 141, 187]

In [479]:
def wcn_value(one, two, graph):
    """
        Funcion que devuelve el valor wcn en comun de esos dos nodos
    """
    cn_list = nx.common_neighbors(graph, one, two)
    
    value_wcn = sum([graph[one][x]['weight'] + graph[two][x]['weight'] for x in cn_list])
    
    return value_wcn


In [480]:
def getValueSimilarMetric(row, df_users_recommend, graph):
    """
        Funcion que va a devolver por cada fila el valor de similaridad entre los dos usuarios de esa fila
    """
    
    return wcn_value(row['user_id'], row['neighbors'], graph)

In [481]:
# ahora para cada lista de de usuarios, hacer una lista de los problemas realizados por esos usuarios, 
# que no los haya realizado ya el usuario
df_separation['sim_value'] = df_separation.apply (lambda row: getValueSimilarMetric(row, df_users_recommend, graph), axis=1)

print(df_separation)

       user_id  neighbors                                      list_problems  \
0           18       3373                      [62, 502, 509, 105, 307, 166]   
1           18       3403         [39, 2, 6, 8, 10, 13, 33, 139, 35, 44, 49]   
2           18       3422                 [39, 150, 183, 327, 191, 325, 390]   
3           18       3427       [150, 254, 134, 191, 325, 390, 39, 162, 327]   
4           18       3432                      [134, 109, 150, 39, 327, 325]   
5           18       3440                           [134, 150, 39, 327, 325]   
6           18       3452                              [39, 44, 33, 49, 253]   
7           18       3428  [150, 78, 171, 181, 51, 155, 174, 136, 235, 44...   
8           18       3481  [39, 33, 109, 13, 162, 134, 159, 251, 27, 29, ...   
9           18       3485                    [162, 39, 33, 44, 49, 254, 183]   
10          18       3493   [109, 39, 70, 141, 134, 187, 258, 150, 162, 209]   
11          18       3498               

In [482]:
# ahora voy a borrar la columna neighbors
del df_separation['neighbors']

In [483]:
df_separation

,user_id,list_problems,sim_value
0,18,"[62, 502, 509, 105, 307, 166]",21
1,18,"[39, 2, 6, 8, 10, 13, 33, 139, 35, 44, 49]",32
2,18,"[39, 150, 183, 327, 191, 325, 390]",24
3,18,"[150, 254, 134, 191, 325, 390, 39, 162, 327]",28
4,18,"[134, 109, 150, 39, 327, 325]",22
5,18,"[134, 150, 39, 327, 325]",20
6,18,"[39, 44, 33, 49, 253]",20
7,18,"[150, 78, 171, 181, 51, 155, 174, 136, 235, 44...",84
8,18,"[39, 33, 109, 13, 162, 134, 159, 251, 27, 29, ...",33
9,18,"[162, 39, 33, 44, 49, 254, 183]",24


In [484]:
# ahora voy a ordenar por el valor de wcn agrupando por el usuario
df_separation = df_separation.sort_values(by=['user_id', 'sim_value'], ascending=False)
print(df_separation)

       user_id                                      list_problems  sim_value
37120     3969  [33, 51, 60, 70, 83, 181, 187, 206, 254, 312, ...       1541
37098     3969  [178, 251, 224, 159, 166, 139, 256, 33, 119, 1...       1521
37334     3969  [507, 33, 83, 217, 312, 256, 49, 4, 503, 254, ...       1446
37097     3969  [53, 119, 251, 256, 224, 253, 181, 155, 143, 1...       1434
37109     3969  [250, 124, 249, 100, 4, 122, 57, 307, 311, 308...       1409
37201     3969  [65, 241, 239, 105, 231, 269, 275, 331, 257, 1...       1263
37240     3969  [254, 253, 2, 251, 315, 250, 252, 257, 49, 390...       1186
37116     3969  [309, 237, 187, 209, 70, 159, 95, 231, 235, 20...       1166
37286     3969  [44, 141, 62, 33, 29, 171, 109, 178, 226, 51, ...       1155
37406     3969  [441, 237, 327, 438, 510, 33, 187, 147, 155, 1...       1132
37055     3969  [6, 13, 93, 2, 4, 8, 15, 17, 19, 23, 27, 29, 3...       1123
37430     3969  [488, 486, 484, 485, 489, 487, 2, 505, 502, 50...       1094

In [485]:
df_separation[df_separation['user_id'] == 1619]

,user_id,list_problems,sim_value


In [486]:
# ahora voy a borrar la columna de similitud por que ya no me hace falta
del df_separation['sim_value']

In [487]:
df_separation

,user_id,list_problems
37120,3969,"[33, 51, 60, 70, 83, 181, 187, 206, 254, 312, ..."
37098,3969,"[178, 251, 224, 159, 166, 139, 256, 33, 119, 1..."
37334,3969,"[507, 33, 83, 217, 312, 256, 49, 4, 503, 254, ..."
37097,3969,"[53, 119, 251, 256, 224, 253, 181, 155, 143, 1..."
37109,3969,"[250, 124, 249, 100, 4, 122, 57, 307, 311, 308..."
37201,3969,"[65, 241, 239, 105, 231, 269, 275, 331, 257, 1..."
37240,3969,"[254, 253, 2, 251, 315, 250, 252, 257, 49, 390..."
37116,3969,"[309, 237, 187, 209, 70, 159, 95, 231, 235, 20..."
37286,3969,"[44, 141, 62, 33, 29, 171, 109, 178, 226, 51, ..."
37406,3969,"[441, 237, 327, 438, 510, 33, 187, 147, 155, 1..."


In [488]:
# hago primero la agrupacion por usuario
grouped_r = df_separation.groupby('user_id')

# hago la agregacion en una lista 
df_recommend_final = grouped_r.aggregate(lambda x:list(x))

print(df_recommend_final)

                                             list_problems
user_id                                                   
18       [[178, 251, 224, 159, 191, 183, 166, 139, 256,...
25       [[33, 51, 60, 70, 83, 134, 181, 191, 254, 315,...
60       [[33, 51, 60, 70, 83, 181, 187, 191, 206, 312,...
130      [[206, 254, 49, 23, 264, 128, 272, 124, 259, 2...
414      [[60, 70, 83, 181, 312, 315, 309, 4, 57, 336, ...
443      [[33, 51, 60, 83, 134, 187, 191, 206, 145, 4, ...
448      [[51, 60, 83, 134, 181, 187, 191, 206, 254, 31...
689      [[51, 60, 70, 83, 181, 187, 191, 206, 254, 312...
912      [[33, 51, 60, 70, 134, 181, 187, 191, 206, 254...
935      [[60, 206, 145, 4, 336, 342, 313, 178, 252, 25...
1711     [[309, 311, 259, 252, 224, 25, 374, 383, 53, 3...
1893     [[33, 51, 60, 70, 83, 181, 187, 191, 206, 254,...
1952     [[51, 60, 70, 83, 134, 181, 187, 191, 206, 254...
1955     [[33, 51, 60, 70, 83, 134, 181, 187, 191, 206,...
2025     [[33, 51, 60, 70, 83, 134, 181, 187, 191, 206,.

In [489]:
def concatenateLists(l):
    """
        Funcion auxiliar para concatenar listas que estan dentro de una lista
    """
    size = len(l)
    
    result = list()
    
    for i in range(0, size):
        value = l[i]
        result = result + value
    
    return result

In [490]:
def concatenateListsRecom(row):
    """
        Funcion que crea una lista de la concatenacion de listas
    """
    
    value = concatenateLists(row['list_problems'])
    
    return value

In [491]:
# ahora para cada lista de de usuarios, hacer una lista de los problemas realizados por esos usuarios, 
# que no los haya realizado ya el usuario
df_recommend_final['recommendation'] = df_recommend_final.apply (lambda row: concatenateListsRecom(row), axis=1)

print(df_recommend_final)

                                             list_problems  \
user_id                                                      
18       [[178, 251, 224, 159, 191, 183, 166, 139, 256,...   
25       [[33, 51, 60, 70, 83, 134, 181, 191, 254, 315,...   
60       [[33, 51, 60, 70, 83, 181, 187, 191, 206, 312,...   
130      [[206, 254, 49, 23, 264, 128, 272, 124, 259, 2...   
414      [[60, 70, 83, 181, 312, 315, 309, 4, 57, 336, ...   
443      [[33, 51, 60, 83, 134, 187, 191, 206, 145, 4, ...   
448      [[51, 60, 83, 134, 181, 187, 191, 206, 254, 31...   
689      [[51, 60, 70, 83, 181, 187, 191, 206, 254, 312...   
912      [[33, 51, 60, 70, 134, 181, 187, 191, 206, 254...   
935      [[60, 206, 145, 4, 336, 342, 313, 178, 252, 25...   
1711     [[309, 311, 259, 252, 224, 25, 374, 383, 53, 3...   
1893     [[33, 51, 60, 70, 83, 181, 187, 191, 206, 254,...   
1952     [[51, 60, 70, 83, 134, 181, 187, 191, 206, 254...   
1955     [[33, 51, 60, 70, 83, 134, 181, 187, 191, 206,...   
2025    

In [492]:
# ahora voy a borrar la columna de similitud por que ya no me hace falta
del df_recommend_final['list_problems']

In [493]:
def delRepetitions(row):
    """
        Funcion auxiliar para evitar que salgan repeticiones en las recomendaciones. Saco la lista de posibles 
        recomendaciones con valores unicos
    """
    conjunto_vacio = set()
    
    # esto sirve para que se haga mas rapido la comprobacion de si el elemento esta en la lista o no
    function_add = conjunto_vacio.add
    
    # hago la lista intensional, para mantener el orden dado en la lista original
    return [x for x in row['recommendation'] if not (x in conjunto_vacio or function_add(x))]

In [494]:
# ahora voy a eliminar las repeticiones
df_recommend_final['recommendation_unique'] = df_recommend_final.apply(lambda row: delRepetitions(row), axis=1)

print(df_recommend_final)

                                            recommendation  \
user_id                                                      
18       [178, 251, 224, 159, 191, 183, 166, 139, 256, ...   
25       [33, 51, 60, 70, 83, 134, 181, 191, 254, 315, ...   
60       [33, 51, 60, 70, 83, 181, 187, 191, 206, 312, ...   
130      [206, 254, 49, 23, 264, 128, 272, 124, 259, 22...   
414      [60, 70, 83, 181, 312, 315, 309, 4, 57, 336, 3...   
443      [33, 51, 60, 83, 134, 187, 191, 206, 145, 4, 1...   
448      [51, 60, 83, 134, 181, 187, 191, 206, 254, 312...   
689      [51, 60, 70, 83, 181, 187, 191, 206, 254, 312,...   
912      [33, 51, 60, 70, 134, 181, 187, 191, 206, 254,...   
935      [60, 206, 145, 4, 336, 342, 313, 178, 252, 25,...   
1711     [309, 311, 259, 252, 224, 25, 374, 383, 53, 34...   
1893     [33, 51, 60, 70, 83, 181, 187, 191, 206, 254, ...   
1952     [51, 60, 70, 83, 134, 181, 187, 191, 206, 254,...   
1955     [33, 51, 60, 70, 83, 134, 181, 187, 191, 206, ...   
2025    

In [495]:
del df_recommend_final['recommendation']

In [496]:
df_recommend_final

,recommendation_unique
user_id,
18,"[178, 251, 224, 159, 191, 183, 166, 139, 256, ..."
25,"[33, 51, 60, 70, 83, 134, 181, 191, 254, 315, ..."
60,"[33, 51, 60, 70, 83, 181, 187, 191, 206, 312, ..."
130,"[206, 254, 49, 23, 264, 128, 272, 124, 259, 22..."
414,"[60, 70, 83, 181, 312, 315, 309, 4, 57, 336, 3..."
443,"[33, 51, 60, 83, 134, 187, 191, 206, 145, 4, 1..."
448,"[51, 60, 83, 134, 181, 187, 191, 206, 254, 312..."
689,"[51, 60, 70, 83, 181, 187, 191, 206, 254, 312,..."
912,"[33, 51, 60, 70, 134, 181, 187, 191, 206, 254,..."


In [497]:
def getKrecomFinal(row, k):
    """
        Funcion que saca las k mejores recomendaciones para el usuario
        Lo que hace es coger los primeros k valores de la lista de recomendaciones
    """
    if k == 1:
        value = list()
        value.append(row['recommendation_unique'][:k])
        return value
    else:
        return row['recommendation_unique'][:k]

In [498]:
k = 10
# ahora saco los k mejores problemas para cada usuario
df_recommend_final['k_recommendation'] = df_recommend_final.apply(lambda row: getKrecomFinal(row, k), axis=1)

print(df_recommend_final)

                                     recommendation_unique  \
user_id                                                      
18       [178, 251, 224, 159, 191, 183, 166, 139, 256, ...   
25       [33, 51, 60, 70, 83, 134, 181, 191, 254, 315, ...   
60       [33, 51, 60, 70, 83, 181, 187, 191, 206, 312, ...   
130      [206, 254, 49, 23, 264, 128, 272, 124, 259, 22...   
414      [60, 70, 83, 181, 312, 315, 309, 4, 57, 336, 3...   
443      [33, 51, 60, 83, 134, 187, 191, 206, 145, 4, 1...   
448      [51, 60, 83, 134, 181, 187, 191, 206, 254, 312...   
689      [51, 60, 70, 83, 181, 187, 191, 206, 254, 312,...   
912      [33, 51, 60, 70, 134, 181, 187, 191, 206, 254,...   
935      [60, 206, 145, 4, 336, 342, 313, 178, 252, 25,...   
1711     [309, 311, 259, 252, 224, 25, 374, 383, 53, 34...   
1893     [33, 51, 60, 70, 83, 181, 187, 191, 206, 254, ...   
1952     [51, 60, 70, 83, 134, 181, 187, 191, 206, 254,...   
1955     [33, 51, 60, 70, 83, 134, 181, 187, 191, 206, ...   
2025    

In [499]:
del df_recommend_final['recommendation_unique']

In [500]:
df_recommend_final

,k_recommendation
user_id,
18,"[178, 251, 224, 159, 191, 183, 166, 139, 256, 33]"
25,"[33, 51, 60, 70, 83, 134, 181, 191, 254, 315]"
60,"[33, 51, 60, 70, 83, 181, 187, 191, 206, 312]"
130,"[206, 254, 49, 23, 264, 128, 272, 124, 259, 226]"
414,"[60, 70, 83, 181, 312, 315, 309, 4, 57, 336]"
443,"[33, 51, 60, 83, 134, 187, 191, 206, 145, 4]"
448,"[51, 60, 83, 134, 181, 187, 191, 206, 254, 312]"
689,"[51, 60, 70, 83, 181, 187, 191, 206, 254, 312]"
912,"[33, 51, 60, 70, 134, 181, 187, 191, 206, 254]"


In [501]:
# ahora tengo que filtrar df_users_eval para que solo contenga las filas de los usuarios a los que hay que recomendar

df_users_eval_filter = df_users_eval[df_users_eval['user_id'].isin(user_list_to_recommend)]
print(df_users_eval_filter)

                                       list_problem_id  user_id
4               [241, 128, 124, 256, 404, 469, 70, 86]       18
5                    [33, 29, 254, 308, 445, 253, 166]       25
8    [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...       60
13                       [254, 253, 308, 166, 49, 445]      130
22   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...      414
26             [134, 187, 256, 251, 90, 174, 147, 438]      443
27   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...      448
34                     [327, 23, 150, 349, 336, 6, 49]      689
44   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...      912
46          [95, 383, 262, 122, 44, 81, 252, 145, 336]      935
70   [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...     1711
73                       [471, 469, 355, 95, 243, 436]     1893
78         [374, 195, 109, 436, 95, 10, 446, 340, 379]     1952
80                            [44, 213, 136, 162, 243]     1955
90             [136, 253, 187, 342, 203,

In [502]:
list_eval_problems = df_users_eval_filter['list_problem_id'].tolist()
list_recom_problems = df_recommend_final['k_recommendation'].tolist()


# meto toda la informacion en un dataframe para obtener las metricas
set_df_metric = {'user_id': user_list_to_recommend, 'eval_problems': list_eval_problems, 'recom_problems': list_recom_problems}
metric_df = pd.DataFrame.from_dict(set_df_metric)

print(metric_df)

                                        eval_problems  \
0              [241, 128, 124, 256, 404, 469, 70, 86]   
1                   [33, 29, 254, 308, 445, 253, 166]   
2   [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...   
3                       [254, 253, 308, 166, 49, 445]   
4   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...   
5             [134, 187, 256, 251, 90, 174, 147, 438]   
6   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...   
7                     [327, 23, 150, 349, 336, 6, 49]   
8   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...   
9          [95, 383, 262, 122, 44, 81, 252, 145, 336]   
10  [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...   
11                      [471, 469, 355, 95, 243, 436]   
12        [374, 195, 109, 436, 95, 10, 446, 340, 379]   
13                           [44, 213, 136, 162, 243]   
14            [136, 253, 187, 342, 203, 33, 441, 508]   
15  [136, 253, 187, 33, 441, 342, 209, 162, 155, 4...   
16                           [1

In [503]:
def one_hit(row):
    """
        Funcion que implementa la metrica one hit. Devuelve un 1 si para un usuarios dado, al menos uno
        de los problemas que se le ha recomendado ha sido realizado por ese usuario en el evaluation_set. 
        Cero si no hay ningun problema de los recomendados que haya sido realizado por el usuario
    """
    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    if len(num_problems_common) >= 1:
        return 1
    else:
        return 0

In [504]:
def mrr(row): 
    """
        Funcion que va a implementar la metrica de evaluacion mrr:
        mrr = 1/ranki, donde ranki es la posicion del primer item correcto
    """

    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    if len(num_problems_common) >= 1:

        # hago la busqueda del primer elemento que esta en la lista de recomendados
        fst_correct_item = -1
        encontrado = False
        i = 0
        while (i < len(row['recom_problems'])) and (encontrado == False):
            if row['recom_problems'][i] in row['eval_problems']:
                # fst_correct_item = row['recom_problems'][i]
                # print(fst_correct_item)
                ranki = i + 1
                encontrado = True
            else:
                i = i + 1
                
        return (1/ranki)

    else:
        return 0

In [505]:
def precision(row):
    """
        Funcion que va a implementar la metrica precision en k: 
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los recomendados
    """
    
    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row['recom_problems']))

In [506]:
def recall(row):
    """
        Funcion que implementa la metrica recall
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los evaluados
    """
    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row['eval_problems']))

In [507]:
def f1(row):
    """
        Funcion que calcula el f1 en funcion de precision y recall
    """
    denominador = row['precision'] + row['recall']
    
    if denominador == 0:
        return 0
    else:
        return (2 * row['precision'] * row['recall']) / denominador

In [508]:
# ahora voy a calcular una metrica para cada usuario
metric_df['one_hit'] = metric_df.apply(lambda row: one_hit(row), axis=1)
metric_df['mrr'] = metric_df.apply(lambda row: mrr(row), axis=1)
metric_df['precision'] = metric_df.apply(lambda row: precision(row), axis=1)
metric_df['recall'] = metric_df.apply(lambda row: recall(row), axis=1)
metric_df['f1'] = metric_df.apply(lambda row: f1(row), axis=1)
print(metric_df)

                                        eval_problems  \
0              [241, 128, 124, 256, 404, 469, 70, 86]   
1                   [33, 29, 254, 308, 445, 253, 166]   
2   [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...   
3                       [254, 253, 308, 166, 49, 445]   
4   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...   
5             [134, 187, 256, 251, 90, 174, 147, 438]   
6   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...   
7                     [327, 23, 150, 349, 336, 6, 49]   
8   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...   
9          [95, 383, 262, 122, 44, 81, 252, 145, 336]   
10  [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...   
11                      [471, 469, 355, 95, 243, 436]   
12        [374, 195, 109, 436, 95, 10, 446, 340, 379]   
13                           [44, 213, 136, 162, 243]   
14            [136, 253, 187, 342, 203, 33, 441, 508]   
15  [136, 253, 187, 33, 441, 342, 209, 162, 155, 4...   
16                           [1

In [509]:
# calculo la media de las metricas

result_one_hit = metric_df['one_hit'].mean()
result_precision = metric_df['precision'].mean()
result_mrr = metric_df['mrr'].mean()
result_recall = metric_df['recall'].mean()
result_f1 = metric_df['f1'].mean()

print("One hit ----------")
print(result_one_hit)
print("Precision ----------")
print(result_precision)
print("Mrr  ----------")
print(result_mrr)
print("Recall  ----------")
print(result_recall)
print("F1  ----------")
print(result_f1)

One hit ----------
0.6
Precision ----------
0.11999999999999995
Mrr  ----------
0.33897435897435896
Recall  ----------
0.11840564606457345
F1  ----------
0.10942753268320728


In [510]:

f = open("C:/hlocal/TFM/nodos_usuarios3", 'a')
f.write(str(result_one_hit) + '\t' + str(result_precision) + '\t' + str(result_mrr) + '\t' + str(result_recall) + '\t' +  str(result_f1) + '\n') 
f.close()